In [ ]:
'''Importing necessary libraries'''
import string #To split the string into words
import pandas
import re #To keep only letters and remove any symbols

In [ ]:
'''Function to remove punctuation marks and symbols'''
def clean_string(input_string):
    #Remove all apostrophes
    cleaned_string = input_string.replace("'", "")
    
    # Replace symbols with spaces
    cleaned_string = re.sub(r'[^a-zA-Z ]', ' ', cleaned_string)

    # Replace multiple spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)

    return cleaned_string.strip()  #Remove leading and trailing spaces

In [ ]:
'''Load letter values from the values.txt file'''
def load_values(filename):  
    values = {}
    with open(filename, 'r') as file:
        for line in file:
            letter, value = line.strip().split()
            values[letter] = int(value)
    return values

In [ ]:
'''Calculate the score for a letter based on its position and value'''
def calculate_score(letter, letter_position, is_last, values):

    letter = letter.upper()  # Convert letter to uppercase
    if letter_position == 1:
        return 0  # First letter of a word has score 0
    elif is_last:
        if letter == 'E':
            return 20  # Last letter is 'E'
        else:
            return 5  # Other last letters
    else:
        position_value = 1 if letter_position == 2 else (2 if letter_position == 3 else 3) #Position value scores
        
        return position_value + values.get(letter, 0)  #Used get to handle missing letters


In [ ]:
'''This function is useful in case a name has multiple words
The letter position will be counted from the last space encountered
So that the letter position is counted from the start of the letter of the word it is in and not the start of the whole name'''
def position_from_last_space(lst, index):
    result = index
    for i in range(index - 1, -1, -1):
        if lst[i] == ' ':
            result = index-i-1

    return result

In [ ]:
"""Generate three-letter abbreviations for a given name."""
def generate_abbreviations(name, values):
    
    #Declaring an empty dictionary. 
    abbreviations = {} 
    
    #Removing all punctuation marks and symbols
    name = clean_string(name)
    
    #Declaring first letter of the abbreviation
    first_letter = name[0].upper() #Converting to uppercase for abbreviation
    
    #Declaring some variables to be used in the loop
    second_letter = ''
    second_letter_position = 0
    second_letter_score = 0
    
    for i in range(1, len(name)-1): #Looping through the letters of each word
        if name[i]!=' ': #Only if the character is not a space, the following statements will be run
            second_letter = name[i].upper()
            second_letter_position = position_from_last_space(name, i) + 1
            second_letter_score = calculate_score(second_letter, second_letter_position, name[i+1]==' ', values) 
            #name[i+1] is a condition to check if the next character of the current element is a space.
            #which tells us if the current element or letter is the last one of the current word.
        else: continue
            
        for j in range(i + 1, len(name)): #Looping to find the 3rd letter of the abbreviation
            if name[j]!=' ': 
                third_letter = name[j].upper()
                
                #Concatenating all letters to create abbreviation
                abbreviation = f"{first_letter}{second_letter}{third_letter}"
                
                third_letter_position = position_from_last_space(name, j) + 1
                
                #Is it the last letter? This is checked ny checking if we encounter a space after the letter
                #or if it is the last letter of the whole name
                is_j_last = True if j==len(name)-1 else (True if name[j+1]==' ' else False) 
                
                third_letter_score = calculate_score(third_letter, third_letter_position, is_j_last, values)
                
                total_score = second_letter_score + third_letter_score #Total score for abbreviation
                abbreviations[abbreviation] = total_score
            else: continue
                
    return abbreviations 

In [ ]:
'''Function to remove duplicate abbreviations'''
def remove_duplicate_abbs(names_abbs_dict):
    #Creating empty sets to store unique abbreviaiotns as sets do not allow duplicate values 
    unique_abbs = set()
    abbs_to_remove = set()

    # Iterate through the outer dictionary
    for name, abbs_and_score in names_abbs_dict.items():
        #Check if there are no abbreviations
        if not abbs_and_score: 
            # Iterate through the inner dictionary
            for abb in abbs_and_score:
                # Check if the inner key is already encountered
                if abb in unique_abbs:
                    abbs_to_remove.add(abb)
                else:
                    unique_abbs.add(abb)

    # Remove duplicate inner keys from the entire dictionary
    for name in names_abbs_dict:
        names_abbs_dict[name] = {ab: score for ab, score in names_abbs_dict[name].items() if ab not in abbs_to_remove}

    return names_abbs_dict


In [ ]:
'''Function for keeping the abbreviations with minimum scores '''
def keep_min_score(names_abbs_dict):
    result_dict = {}

    for name, abbs_and_score in names_abbs_dict.items():
        min_score = min(abbs_and_score.values())
        min_abbs = [abb for abb, score in abbs_and_score.items() if score == min_score]
        result_dict[name] = {min_abb: min_score for min_abb in min_abbs}

    return result_dict

In [ ]:
'''Main function'''
def main():
    #Getting input file and output filename
    input_filename = input("Enter the input filename (e.g., names.txt): ")
    output_filename = input("Enter your surname (for output filename): ").lower() + f"_{input_filename}_abbrevs.txt"

    #Loading the contents of values file
    values = load_values("values.txt")
    
    #Empty dictionary for all abbreviations
    all_abbreviations = {}

    #Opening the input file in read mode and output file in write mode
    with open(input_filename, 'r') as input_file, open(output_filename, 'w') as output_file:
        
        for line in input_file:
            name = line.strip() #Splitting each line into names
            
            abbreviations = generate_abbreviations(name, values)
            if not abbreviations:
                output_file.write('\n')  #No abbreviation found for the name
            else:
                best_abbreviation, best_score = min(abbreviations.items(), key=lambda x: x[1])
                #This line finds out the abbreviation with min score with its score
                #Uses the min function where key is the argument based on which the min value will be found
                #Here key will be the score of each abbreviation which is value part of the pair in the abbreviation set.
                
                #Storing all abbreviations in the dictionary
                all_abbreviations[name] = abbreviations
                
                #Removing duplicate abbreviations
                all_abbbreviations = remove_duplicate_abbs(all_abbreviations)
                
        #Keeping abbreviations with minimum scores
        all_abbreviations = keep_min_score(all_abbreviations)
        
        #Writing the abbreviations beside the name in a file
        for key_name in all_abbreviations:
            output_file.write(key_name + ' | ')
            output_file.write(' '.join(all_abbreviations[key_name]) + '\n') #Joining the best abbreviations
                

if __name__ == "__main__":
    main()
